In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LassoCV,Lasso
from sklearn.metrics import mean_squared_error

In [2]:

df = pd.read_csv('world-happiness-report-2021.csv')
#df.head()
#df.info()
#df.describe()

df2 = pd.read_csv('world-happiness-report.csv')
df2.head()

#Création liste des régions du monde par pays
country_region = df.iloc[:,0:2]

#Mise en forme des 2 datasets en prévision du concat
#dataset de 2021 :
df_1 = df.iloc[:,:12]
df_1 = df_1.drop(df.iloc[:,3:6], axis=1)
df_1 = df_1.drop('Regional indicator', axis=1)
df_1['year']='2021'

#dataset historique depuis 2005 :
df_2 = df2.iloc[:, :-2]
dico = {'Life Ladder' : 'Ladder score', 'Log GDP per capita' : 'Logged GDP per capita', 'Healthy life expectancy at birth' : 'Healthy life expectancy'}
df_2.rename(dico, axis=1, inplace=True)

#création du fichier contenants les données depuis 2005 :
df_full = pd.concat([df_1, df_2], axis=0)
df_full['year'] = df_full['year'].astype('int')
df_full = df_full.sort_values(by='Country name', ascending=True)

#ajout colonne Region d'après le df country_region créé précédemment :
df_full = df_full.merge(country_region, on = 'Country name', how ='inner')

#ajout du rank par année de chaque pays
df_full["rank"] = df_full.groupby("year")["Ladder score"].rank("dense", ascending=False).astype('int')

df_full.head()
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2035 entries, 0 to 2034
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Country name                  2035 non-null   object 
 1   Ladder score                  2035 non-null   float64
 2   Logged GDP per capita         2011 non-null   float64
 3   Social support                2026 non-null   float64
 4   Healthy life expectancy       1984 non-null   float64
 5   Freedom to make life choices  2005 non-null   float64
 6   Generosity                    1959 non-null   float64
 7   Perceptions of corruption     1931 non-null   float64
 8   year                          2035 non-null   int32  
 9   Regional indicator            2035 non-null   object 
 10  rank                          2035 non-null   int32  
dtypes: float64(7), int32(2), object(2)
memory usage: 174.9+ KB


In [3]:

#Remplacement des NaN par les moyennes des variables en fonction de chaque pays
for i in df_full.iloc[:,2:8].columns : 
    df_full[i] = df_full[i].fillna(df_full.groupby('Country name')[i].transform("mean"))

In [4]:
def filtre(x):
  if x >= 0 and x < 5 :
    return '3'
  if x >= 5 and x <= 6 :
    return '2'
  if x > 6 and x <= 10 :
    return '1'

df_full['Ladder score_categ'] = df_full['Ladder score'].apply(filtre)

In [ ]:
df_full.shape

In [5]:
## variable cile

target = df_full['Ladder score_categ']

#df pour la rl

col_feat = ['Logged GDP per capita', 'Social support', 'Healthy life expectancy',
       'Freedom to make life choices', 'Generosity',
       'Perceptions of corruption']



rlm_df_full = df_full[col_feat]

In [6]:
#Standardisation des données


scaler = StandardScaler()
data = scaler.fit_transform(rlm_df_full)



In [7]:
# Séparation des données en deux jeux train et test:

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=10)


In [16]:
l = np.arange(100,1000,10)
print(l)


[100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270
 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450
 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630
 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800 810
 820 830 840 850 860 870 880 890 900 910 920 930 940 950 960 970 980 990]


In [17]:
clf = RandomForestClassifier()

param = {"max_features" : ["sqrt", "log2"], "n_estimators": l, "criterion" :["gini", "entropy"]}

grid = GridSearchCV(estimator=clf,param_grid=param)
grid.fit(X_train,y_train)
print(grid.best_params_)


{'criterion': 'entropy', 'max_features': 'sqrt', 'n_estimators': 220}


In [20]:
clf_best = RandomForestClassifier(criterion="entropy", max_features="sqrt", n_estimators = 220)

clf_best.fit(X_train,y_train)
y_pred = clf_best.predict(X_test)

print('score test :', clf_best.score(X_test, y_test))
print('mse test :', mean_squared_error(y_pred, y_test))

score test : 0.8329238329238329
mse test : 0.17444717444717445


In [21]:
pd.crosstab(y_test, y_pred, rownames = ['Classe réelle'], colnames = ['Classe prédite'])

Classe prédite,1,2,3
Classe réelle,,,
1,115,19,0
2,13,103,19
3,1,16,121


In [22]:
from sklearn.metrics import classification_report

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.89      0.86      0.87       134
           2       0.75      0.76      0.75       135
           3       0.86      0.88      0.87       138

    accuracy                           0.83       407
   macro avg       0.83      0.83      0.83       407
weighted avg       0.83      0.83      0.83       407

